In [1]:
# Nonlinear MPC test using kinematic bicycle model for car
# We want to use the kinematic bicycle model from this paper (Figure 3)
# https://link-springer-com.stanford.idm.oclc.org/article/10.1007/s13177-020-00226-1#Tab4

# which has states:
# x, y = positional coordinates in inertial frame
# ψ = angle of car centerline in inertial frame
# ̇ψ = change of angle
# β = angle between car centerline and velocity vector
# ̇β = change of angle

# and has the equations:
# ̇x = v cos(ψ + β)
# ̇y = v sin(ψ + β)
# ̇v = a
# ̇ψ = v / lᵣ sin(β)
# β = arctan(lᵣ/(lf + lᵣ) tan(δf))
# ̇β = v/(lf + lᵣ) tan(δf) - v/lᵣ sin(β)
# ̇δf = ω

# and we want to solve it. Define the model:


The model has state and input:
\begin{gather*}
z = \begin{bmatrix}x\\ y\\ v\\ \psi\end{bmatrix},\quad
u = \begin{bmatrix}a\\ \delta_f\end{bmatrix},\quad
\end{gather*}
where the two control signals are $a$, the acceleration command, and $\delta_f$, the front steering angle (in radians).
The nonlinear model is:
\begin{gather*}
\begin{bmatrix}
\dot x\\ \dot y\\ \dot v\\ \dot\psi
%\\ \dot\beta
\end{bmatrix} =
\begin{bmatrix}
v\cos(\psi + \beta)
\\
v\sin(\psi + \beta)
\\
a
\\
\frac{v}{l_r}\sin(\beta)
%\\
%\frac{v}{l_f + l_r}\tan(\delta_f) - \frac{v}{l_r}\sin(\beta)
\end{bmatrix}
\end{gather*}

## Casadi setup
From http://casadi.sourceforge.net/v3.4.4/users_guide/casadi-users_guide.pdf

In [2]:
# Library dependencies
import casadi
import numpy             as np
import matplotlib.pyplot as plt
import matplotlib        as mpl
import bezier

# Local dependencies
from direct_collocation  import MpcProblem
from KinematicBicycleCar import KinematicBicycleCar
from roadrunner_2        import Roadrunner, Segment
from road                import iso_double_lane_change as test_road

## Initial state and cost

In [3]:
# Initialize the car model with 0.05 second steps and 30 step lookahead
car = KinematicBicycleCar(N=30, step=0.075)

DESIRED_SPEED = 5.0

In [4]:
n_test_road_pts,_ = np.shape(test_road)

roadrunner = Roadrunner(road_center = test_road,                    # Road centerline points
                        road_width  = np.ones(n_test_road_pts)*5.0, # constant lane width of 5 meters
                        P           = 10) # Number of points to try curve-fitting at a time



In [5]:
# Boundary functions

# terrible hack, need to fix right away

global dist_already_traveled
dist_already_traveled = 0.0

def road_center(model:KinematicBicycleCar, k:int)->np.array:
    global dist_already_traveled

    (xy, angle, _) = roadrunner.evaluate(model.step*np.sum(model.state_estimate[2,:k]), full_data=True)

    center = np.empty((3,))
    center[0:2] = np.reshape(xy, 2)
    center[2]   = angle
    return center

# Test

def bound_x(model:KinematicBicycleCar, k:int)->[np.array]:
    # Returns a 4-sided polygon bound, like this:
    # x2-------x3
    #   \  o  /
    #  x1-----x4
    # successive bounds will overlap each other
    # so there is freedom to slow down or speed up the vehicle.
    
    dist_behind = (k-5)*model.step*DESIRED_SPEED # 5 steps behind * timestep * velocity at point k
    #print("Looking behind by", dist_behind)
    (center_minus, angle_minus, width_minus) = roadrunner.evaluate(dist_behind, full_data=True)
    
    dist_ahead = (k+5)*model.step*DESIRED_SPEED # 5 steps ahead * timestep * velocity at point k
    #print("Looking ahead by", dist_ahead)
    (center_plus, angle_plus, width_plus) = roadrunner.evaluate(dist_ahead, full_data=True)

    center_minus = np.reshape(center_minus,(2,))
    center_plus = np.reshape(center_plus,(2,))
    
    #print("Got center points", center_minus, center_plus)
    #print("center angles:", 180.0/np.pi*(angle_minus), 180.0/np.pi*(angle_plus))
    # These four points define the 2d bounding box for the car at state xk
    
    p1 = np.reshape(np.array([center_minus[0] + width_minus/2.0*np.cos(angle_minus-np.pi/2),
                   center_minus[1] + width_minus/2.0*np.sin(angle_minus-np.pi/2)]), (2,))
    
    p2 = np.reshape(np.array([center_minus[0] + width_minus/2.0*np.cos(angle_minus+np.pi/2),
                   center_minus[1] + width_minus/2.0*np.sin(angle_minus+np.pi/2)]), (2,))
    
    p3 = np.reshape(np.array([center_plus[0] + width_plus/2.0*np.cos(angle_plus+np.pi/2),
                   center_plus[1] + width_plus/2.0*np.sin(angle_plus+np.pi/2)]), (2,))
    
    p4 = np.reshape(np.array([center_plus[0] + width_plus/2.0*np.cos(angle_plus-np.pi/2),
                   center_plus[1] + width_plus/2.0*np.sin(angle_plus-np.pi/2)]), (2,))
    
    # The slopes between the points
    slope12 = (p2[1]-p1[1])/(p2[0]-p1[0]); slope12 = 1e4 if np.isnan(slope12) else slope12
    slope23 = (p3[1]-p2[1])/(p3[0]-p2[0]); slope23 = 1e4 if np.isnan(slope23) else slope23
    slope43 = (p3[1]-p4[1])/(p3[0]-p4[0]); slope43 = 1e4 if np.isnan(slope43) else slope43
    slope14 = (p1[1]-p4[1])/(p1[0]-p4[0]); slope14 = 1e4 if np.isnan(slope14) else slope14

    # form 4 linear inequalities to represent the box
    # we either have 0.0 >= ax + by + c >= -Inf
    # or             Inf >= ax + by + c >= 0.0
    
    # since y = (x-x0)*slope + y0
    # use the line 0 = x*slope - y + y0 - x0*slope as the upper / lower bound
    
    #            (upper)
    #     p2-----slope23------p3
    #    /                    /
    # slope12 (upper)   slope34 (lower)
    #  /                   /
    # p1-----slope14-----p4
    #        (lower)
    
    # upper bounds (if slope is positive)
    bounds = [
        np.array([np.inf, slope12, -1.0, (p1[1]-p1[0]*slope12), 0.0 - 0]),
        np.array([np.inf, slope23, -1.0, (p2[1]-p2[0]*slope23), 0.0 - 3]),
    
    # lower bounds (if slope is positive)
        np.array([0.0 + 0, slope43, -1.0, (p4[1]-p4[0]*slope43), -np.inf]),
        np.array([0.0 + 3, slope14, -1.0, (p1[1]-p1[0]*slope14), -np.inf]),
    ]
    
    for i in range(len(bounds)): # Handle the negative-slope flipped cases
    
        if np.sign(bounds[i][1]) == -1: # slope is negative, so inequality flips
            bounds[i][0], bounds[i][4] = -bounds[i][4], -bounds[i][0]
            
    return bounds, np.vstack([p1,p2,p3,p4])


In [6]:
# What we should do now
# 1. Use the velocity estimate to "step through" the road,
#    obtaining a set of road midpoints and constraint boundaries
# 2. Add a term to the cost function that attracts the successive states to the road midpoints - done
# 3. Use the midpoints to initialize the variable estimate so the solver is faster. - done,
#    and we are initializing the new control uk to the previously computed one
#    kind of warm-starting ipopt - done


## Run the MPC algorithm

In [ ]:
roadrunner.reset()
# start a bit ahead of the first point to have space for looking back on the road
roadrunner.advance(car.step*7*DESIRED_SPEED)

xy = np.reshape(roadrunner.evaluate(),(2,))
ic = [xy[0], xy[1], DESIRED_SPEED, float(roadrunner.get_angle())]
car.set_initial(ic)
print("Starting at initial conditions", ic)

# This is the model:
# https://link-springer-com.stanford.idm.oclc.org/article/10.1007/s13177-020-00226-1/figures/3
mpcprob = MpcProblem(car,     # casadi.DaeBuilder()
                     bound_x=bound_x,
                     road_center=road_center,
                    )


# Controls the NUMBER of iterations to run the MPC algorithm.
n_runs = 350

x_plan = np.empty((mpcprob.model.n,n_runs+1)) # store the steps that get executed (n_runs)
u_plan = np.empty((mpcprob.model.m,n_runs))   # store the control inputs that get executed
x_true = np.empty((mpcprob.model.n,n_runs+1)) # store the state as simulated for each control input by an integrator
x_true[:,0] = ic # initial conditions
x_plan[:,0] = ic
# store the polygon boundary for each step, so we can plot them later
polygon_boundaries = np.zeros((n_runs+mpcprob.model.N,4,2))

for k in range(n_runs):
    # Solve the MPC problem
    xk_opt, uk_opt, sol = mpcprob.run(x_true[:,k])
    
    # Save the next state x_k+1 and control u_k+1
    for i in range(mpcprob.model.n):
        x_plan[i,k+1] = xk_opt[i][1]
    for i in range(mpcprob.model.m):
        u_plan[i,k] = uk_opt[i][1]
    x_true[:,k+1] = mpcprob.simulate(x_true[:,k], u_plan[:,k])
    
    # Compute distance traveled (approximately)
    step_xy = np.linalg.norm(x_true[:,k+1]-x_true[:,k], 2) #np.sqrt((x_true[0,k+1]-x_true[0,k])**2 + (x_true[1,k+1]-x_true[1,k])**2)
    
    # Save the 4 points that represent the polygon boundary at state k
    polygon_boundaries[k,:,:] = mpcprob.p_plot[0,:,:]
    
    roadrunner.advance(step_xy)
    print("Advanced by", step_xy,"to", roadrunner.evaluate())

    # Done!

Starting at initial conditions [7.973393420909675, 0.9523902139751268, 5.0, 0.0022467447187692127]
Attracting  X_1[0] 1 to  [8.35529335e+00 9.53264014e-01 2.24674472e-03]
Attracting  X_2[0] 2 to  [8.73556648e+00 9.54166553e-01 2.24674472e-03]
Attracting  X_3[0] 3 to  [9.11412869e+00 9.55100336e-01 2.24674472e-03]
Attracting  X_4[0] 4 to  [9.49088901e+00 9.56069903e-01 2.24674472e-03]
Attracting  X_5[0] 5 to  [9.86575201e+00 9.57082054e-01 2.24674472e-03]
Attracting  X_6[0] 6 to  [1.02386208e+01 9.58146074e-01 2.24674472e-03]
Attracting  X_7[0] 7 to  [1.06094001e+01 9.59273960e-01 2.24674472e-03]
Attracting  X_8[0] 8 to  [1.09780008e+01 9.60480648e-01 2.24674472e-03]
Attracting  X_9[0] 9 to  [1.13443435e+01 9.61784222e-01 2.24674472e-03]
Attracting  X_10[0] 10 to  [1.17083631e+01 9.63206131e-01 2.24674472e-03]
Attracting  X_11[0] 11 to  [1.20700141e+01 9.64771379e-01 2.24674472e-03]
Attracting  X_12[0] 12 to  [1.24292752e+01 9.66508711e-01 2.24674472e-03]
Attracting  X_13[0] 13 to  [1.2

Attracting  X_30[0] 30 to  [19.0674344   1.16738118  0.06348993]
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:      960
Number of nonzeros in Lagrangian Hessian.............:      568

Total number of variables............................:      540
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      300
                     variables with only upper bounds:        0
Total number of equality constraints.................:      480
Total number of inequality constraints...............:      480
        inequality constraints with only lower bounds:      240
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      240

iter    objective    i

Attracting  X_25[0] 25 to  [17.41431285  1.07855655  0.04434964]
Attracting  X_26[0] 26 to  [17.77631338  1.09533331  0.04829548]
Attracting  X_27[0] 27 to  [18.14355831  1.11384136  0.0524366 ]
Attracting  X_28[0] 28 to  [18.51598918  1.13419505  0.05678105]
Attracting  X_29[0] 29 to  [18.89351338  1.15651247  0.06133773]
Attracting  X_30[0] 30 to  [19.27602066  1.18091629  0.066116  ]
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:      960
Number of nonzeros in Lagrangian Hessian.............:      568

Total number of variables............................:      540
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      300
                     variables with only upper bounds:        0
Total number of eq

Advanced by 0.37252296080802566 to [[9.86054946 0.95706764]]
Attracting  X_1[0] 1 to  [1.02192917e+01 9.58089314e-01 2.76964940e-03]
Attracting  X_2[0] 2 to  [1.05701214e+01 9.59150713e-01 2.76964940e-03]
Attracting  X_3[0] 3 to  [1.09128902e+01 9.60260464e-01 2.76964940e-03]
Attracting  X_4[0] 4 to  [1.12474801e+01 9.61428170e-01 2.76964940e-03]
Attracting  X_5[0] 5 to  [1.15738572e+01 9.62664362e-01 2.76964940e-03]
Attracting  X_6[0] 6 to  [1.18921122e+01 9.63980490e-01 2.76964940e-03]
Attracting  X_7[0] 7 to  [1.22024888e+01 9.65389006e-01 2.76964940e-03]
Attracting  X_8[0] 8 to  [1.2505400e+01 9.6690356e-01 2.7696494e-03]
Attracting  X_9[0] 9 to  [1.28014324e+01 9.68539330e-01 2.76964940e-03]
Attracting  X_10[0] 10 to  [1.30913404e+01 9.70313474e-01 2.76964940e-03]
Attracting  X_11[0] 11 to  [1.33760314e+01 9.72245719e-01 2.76964940e-03]
Attracting  X_12[0] 12 to  [1.36565430e+01 9.74359038e-01 2.76964940e-03]
Attracting  X_13[0] 13 to  [1.39340142e+01 9.76680402e-01 2.76964940e-03

Attracting  X_23[0] 23 to  [17.18787734  1.06878086  0.04194958]
Attracting  X_24[0] 24 to  [17.52587487  1.08357458  0.04555137]
Attracting  X_25[0] 25 to  [17.87313504  1.10006524  0.04937367]
Attracting  X_26[0] 26 to  [18.22968018  1.1184042   0.05342813]
Attracting  X_27[0] 27 to  [18.59546581  1.13875036  0.05772739]
Attracting  X_28[0] 28 to  [18.97040013  1.16127097  0.06228501]
Attracting  X_29[0] 29 to  [19.35436214  1.18614273  0.06711508]
Attracting  X_30[0] 30 to  [19.74721822  1.21355292  0.07223171]
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:      960
Number of nonzeros in Lagrangian Hessian.............:      568

Total number of variables............................:      540
                     variables with only lower bounds:        0
                

...

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:      960
Number of nonzeros in Lagrangian Hessian.............:      568

Total number of variables............................:      540
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      300
                     variables with only upper bounds:        0
Total number of equality constraints.................:      480
Total number of inequality constraints...............:      480
        inequality constraints with only lower bounds:      200
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      280

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Dual infeasibility......:   1.3444800828210646e-13    1.3444800828210646e-13
Constraint violation....:   2.1921353621223716e-13    2.1921353621223716e-13
Complementarity.........:   9.1197834082386086e-10    9.1197834082386086e-10
Overall NLP error.......:   9.1197834082386086e-10    9.1197834082386086e-10


Number of objective function evaluations             = 15
Number of objective gradient evaluations             = 15
Number of equality constraint evaluations            = 15
Number of inequality constraint evaluations          = 15
Number of equality constraint Jacobian evaluations   = 15
Number of inequality constraint Jacobian evaluations = 15
Number of Lagrangian Hessian evaluations             = 14
Total CPU secs in IPOPT (w/o function evaluations)   =      0.041
Total CPU secs in NLP function evaluations           =      0.034

EXIT: Optimal Solution Found.
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 363.00us ( 24.20us) 364.07us ( 24.27us)

Attracting  X_24[0] 24 to  [ 6.66597487e+01  4.69625606e+00 -1.13295828e-02]
Attracting  X_25[0] 25 to  [ 6.71640213e+01  4.68996668e+00 -1.36554726e-02]
Attracting  X_26[0] 26 to  [ 6.76817248e+01  4.68222120e+00 -1.63108392e-02]
Attracting  X_27[0] 27 to  [ 6.82124389e+01  4.67277704e+00 -1.93263032e-02]
Attracting  X_28[0] 28 to  [ 6.87556471e+01  4.66136671e+00 -2.27336248e-02]
Attracting  X_29[0] 29 to  [ 6.93107352e+01  4.64769796e+00 -2.65655479e-02]
Attracting  X_30[0] 30 to  [ 6.98769926e+01  4.63145447e+00 -3.08556006e-02]
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:      960
Number of nonzeros in Lagrangian Hessian.............:      568

Total number of variables............................:      540
                     variables with only lower bounds:       

Attracting  X_22[0] 22 to  [ 6.62011907e+01  4.70100807e+00 -9.42885603e-03]
Attracting  X_23[0] 23 to  [ 6.66826278e+01  4.69599570e+00 -1.14296914e-02]
Attracting  X_24[0] 24 to  [ 6.71778501e+01  4.68977736e+00 -1.37228295e-02]
Attracting  X_25[0] 25 to  [ 6.76865392e+01  4.68214260e+00 -1.63368405e-02]
Attracting  X_26[0] 26 to  [ 6.82082909e+01  4.67285716e+00 -1.93015507e-02]
Attracting  X_27[0] 27 to  [ 6.87426094e+01  4.66166259e+00 -2.26479349e-02]
Attracting  X_28[0] 28 to  [ 6.92889050e+01  4.64827631e+00 -2.64079715e-02]
Attracting  X_29[0] 29 to  [ 6.98464951e+01  4.63239211e+00 -3.06144558e-02]
Attracting  X_30[0] 30 to  [ 7.04146092e+01  4.61368119e+00 -3.53007700e-02]
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:      960
Number of nonzeros in Lagrangian Hes

Objective...............:   6.4436918109856336e+00    6.4436918109856336e+00
Dual infeasibility......:   1.2079226507921703e-13    1.2079226507921703e-13
Constraint violation....:   2.0089485630592208e-13    2.0089485630592208e-13
Complementarity.........:   9.1908046753767969e-10    9.1908046753767969e-10
Overall NLP error.......:   9.1908046753767969e-10    9.1908046753767969e-10


Number of objective function evaluations             = 15
Number of objective gradient evaluations             = 15
Number of equality constraint evaluations            = 15
Number of inequality constraint evaluations          = 15
Number of equality constraint Jacobian evaluations   = 15
Number of inequality constraint Jacobian evaluations = 15
Number of Lagrangian Hessian evaluations             = 14
Total CPU secs in IPOPT (w/o function evaluations)   =      0.044
Total CPU secs in NLP function evaluations           =      0.037

EXIT: Optimal Solution Found.
      solver  :   t_proc      (avg)   t_wall

In [ ]:
# FOR PLOTTING: We also want to include the polygons for the lookahead steps k=1,...,N.
polygon_boundaries[n_runs-1:n_runs+mpcprob.model.N+2,:,:] = mpcprob.p_plot

## Control signals: acceleration and steering angle

In [ ]:
u_plt, u_ax1, u_ax2 = car.plot_u(u_plan, mpcprob.u_opt)

## x-y plot of car's path
Arrows show velocity. Orange components are planned but not executed.

In [ ]:
x_plt, x_ax = car.plot_x(x_plan, mpcprob.x_opt)
    
roadrunner.plot(ax=x_ax)
x_ax.scatter(ic[0], ic[1], color="green")
plt.ylim(-10,10)
plt.show()

## x vs time and y vs time, separately, with the boundaries

In [ ]:
# Plot the road boundary as polygons
def plot_polygons(polygon_boundaries, ax=None):
    n_polys,_,_ = np.shape(polygon_boundaries)
    
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(8,4))
        
    for k in range(n_polys):
        # Plot the polygon fill
        ax.fill(polygon_boundaries[k,:,0], polygon_boundaries[k,:,1], "lightblue", alpha=0.1+(k/(n_polys)/2))
        # Plot the line around each polygon
        ax.plot(np.concatenate([polygon_boundaries[k,:,0],polygon_boundaries[k,0:1,0]]),
                np.concatenate([polygon_boundaries[k,:,1],polygon_boundaries[k,0:1,1]]),
                color="navy", alpha=0.1+k/(n_polys)/2)

    # Plot the last polygon in RED
    ax.plot(np.concatenate([polygon_boundaries[-1,:,0],polygon_boundaries[-1,0:1,0]]),
            np.concatenate([polygon_boundaries[-1,:,1],polygon_boundaries[-1,0:1,1]]),
            color="red", alpha=1)
    return ax

ax = plot_polygons(polygon_boundaries)
ax.scatter(test_road[0:15,0], test_road[0:15,1], color='red')
ax.scatter(ic[0], ic[1], color="green")

In [ ]:
# Test - find a feasible point inside one polygon, to prove they work

i = 10
roadrunner.reset()
roadrunner.advance(mpcprob.model.step*np.sum(mpcprob.model.state_estimate[2,:i]))
bounds, p = bound_x(car, 0)
print("p", p)
x = casadi.MX.sym("x", 2)
lbg = []; ubg = []; g = [];

fig, ax = plt.subplots(1,1)


for (ub, a, b, c, lb) in bounds:
    print("Constraint with lower bound ", lb, "upper ", ub)
    lbg.append(np.reshape(lb,(1,)))
    ubg.append(np.reshape(ub,(1,)))
    g.append(x[0]*a + x[1]*b + c)
xy = roadrunner.evaluate()
print("At point", xy)
xl = np.linspace(xy[0,0]-4, xy[0,0]+4,10)
print(xl)
for (ub, a, b, c, lb) in bounds:
    print("ub, a, b, c, lb", ub, a, b, c, lb)
    if np.isinf(lb): # it is an upper bound
        if np.sign(a) == 1:
            ax.plot(xl, (xl*a + c)/(-b), label="upper"+str(i), color="blue", linestyle='--')
        else:
            ax.plot(xl, (xl*a + c)/(-b), label="upper"+str(i), color="red", linestyle='--')
    else:            # it is a lower bound
        if np.sign(a) == 1:
            ax.plot(xl, (xl*a + c)/(-b), label="lower"+str(i), color="red")
        else:
            ax.plot(xl, (xl*a + c)/(-b), label="lower"+str(i), color="blue")


plt.scatter(p[:,0], p[:,1], color="red")
for k in range(-5,5):
    xy = roadrunner.evaluate(mpcprob.model.step*5*k)
    plt.scatter(xy[0,0], xy[0,1], color="blue")
#for (ub, a, b, c, lb) in lowers:
#    ubg.append(np.array([np.inf,]))
#    lbg.append(np.zeros((1,)))
#    g.append(x[0]*a + x[1]*b + c)

ubg = np.concatenate(ubg)
lbg = np.concatenate(lbg)
g = casadi.vertcat(*g)

J = x[0]**2 + x[1]**2

# Create an NLP solver
prob = {'f': J, 'x': x, 'g': g}
solver = casadi.nlpsol('solver', 'ipopt', prob, {'verbose':False});

# Solve the NLP
sol = solver(x0=[2,1], lbx=-np.inf, ubx=np.inf, lbg=lbg, ubg=ubg)
print(sol['x'])
plt.ylim([-10,20])